In [ ]:
import pandas as pd

# ============ 配置參數 ============
THRESHOLD = 1  # accuracy_reward 的閾值
# ==================================

# 定義所有 CSV 文件
csv_files = [
    'gemini_flash_gt_reasonning_0_170.csv',
    'gemini_flash_gt_reasonning_170_500.csv',
    'gemini_flash_gt_reasonning_500_850.csv',
    'gemini_flash_gt_reasonning_850_1200.csv',
]

# 存儲結果
results = []
total_records = 0
total_good_records = 0

print("="*80)
print(f"📊 開始分析各個 CSV 文件 (閾值: accuracy_reward > {THRESHOLD})")
print("="*80)

for csv_file in csv_files:
    try:
        # 讀取 CSV
        df = pd.read_csv(csv_file, encoding='utf-8')
        
        # 計算統計數據
        total_count = df.shape[0]
        good_count = (df['accuracy_reward'] >= THRESHOLD).sum()
        good_ratio = (good_count / total_count * 100) if total_count > 0 else 0
        
        # 累加總計
        total_records += total_count
        total_good_records += good_count
        
        # 儲存結果
        results.append({
            'file': csv_file,
            'total': total_count,
            'good': good_count,
            'ratio': good_ratio
        })
        
        # 顯示單個文件的結果
        print(f"\n📁 文件: {csv_file}")
        print(f"   總筆數: {total_count}")
        print(f"   accuracy_reward > {THRESHOLD} 的數量: {good_count}")
        print(f"   佔比: {good_ratio:.2f}%")
        
    except FileNotFoundError:
        print(f"\n⚠️  文件未找到: {csv_file}")
    except Exception as e:
        print(f"\n❌ 讀取 {csv_file} 時發生錯誤: {e}")

# 顯示總計統計
print("\n" + "="*80)
print(f"📈 總計統計 (閾值: accuracy_reward > {THRESHOLD})")
print("="*80)
print(f"✅ 總共分析文件數: {len(results)}")
print(f"📊 總資料筆數: {total_records}")
print(f"🎯 accuracy_reward > {THRESHOLD} 的總數量: {total_good_records}")
print(f"📈 總體佔比: {(total_good_records/total_records*100):.2f}%")
print("="*80)

# 顯示詳細表格
print("\n📋 詳細統計表格:")
print("-"*80)
print(f"{'文件名':<40} {'總筆數':>10} {'良好筆數':>10} {'佔比':>10}")
print("-"*80)
for result in results:
    print(f"{result['file']:<40} {result['total']:>10} {result['good']:>10} {result['ratio']:>9.2f}%")
print("-"*80)
print(f"{'總計':<40} {total_records:>10} {total_good_records:>10} {(total_good_records/total_records*100):>9.2f}%")
print("="*80)

📊 開始分析各個 CSV 文件 (閾值: accuracy_reward > 1)

📁 文件: gemini_flash_gt_reasonning_0_170.csv
   總筆數: 170
   accuracy_reward > 1 的數量: 167
   佔比: 98.24%

📁 文件: gemini_flash_gt_reasonning_170_500.csv
   總筆數: 301
   accuracy_reward > 1 的數量: 299
   佔比: 99.34%

📁 文件: gemini_flash_gt_reasonning_500_850.csv
   總筆數: 291
   accuracy_reward > 1 的數量: 286
   佔比: 98.28%

📁 文件: gemini_flash_gt_reasonning_850_1200.csv
   總筆數: 298
   accuracy_reward > 1 的數量: 292
   佔比: 97.99%

📈 總計統計 (閾值: accuracy_reward > 1)
✅ 總共分析文件數: 4
📊 總資料筆數: 1060
🎯 accuracy_reward > 1 的總數量: 1044
📈 總體佔比: 98.49%

📋 詳細統計表格:
--------------------------------------------------------------------------------
文件名                                             總筆數       良好筆數         佔比
--------------------------------------------------------------------------------
gemini_flash_gt_reasonning_0_170.csv            170        167     98.24%
gemini_flash_gt_reasonning_170_500.csv          301        299     99.34%
gemini_flash_gt_reasonning_500_850.csv 

In [14]:
# 先檢查原始 CSV 的結構
print("="*80)
print("🔍 檢查所有 CSV 檔案的欄位結構")
print("="*80)

for csv_file in csv_files[:3]:  # 先檢查前 3 個檔案
    try:
        df = pd.read_csv(csv_file, encoding='utf-8', nrows=5)
        print(f"\n📁 {csv_file}")
        print(f"   欄位: {df.columns.tolist()}")
        if 'index' in df.columns:
            print(f"   前 5 個 index 值: {df['index'].head().tolist()}")
    except Exception as e:
        print(f"   ❌ 讀取錯誤: {e}")

print("\n" + "="*80)

🔍 檢查所有 CSV 檔案的欄位結構

📁 gemini_flash_gt_reasonning_0_170.csv
   欄位: ['index', 'raw_question', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
   前 5 個 index 值: [0, 1, 2, 3, 4]

📁 gemini_flash_gt_reasonning_170_500.csv
   欄位: ['index', 'raw_question', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
   前 5 個 index 值: [170, 171, 172, 173, 174]

📁 gemini_flash_gt_reasonning_500_850.csv
   欄位: ['index', 'raw_question', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
   前 5 個 index 值: [500, 501, 502, 503, 504]



In [15]:
import json

filtered_dfs = []

# 首先讀取 JSONL 檔案，建立 index 到 question 的映射
print("正在讀取 note_icd_data.jsonl...")
index_to_question = {}

try:
    with open('note_icd_data.jsonl', 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            try:
                data = json.loads(line.strip())
                # 假設 JSONL 中每行的 index 與行號對應（從 0 開始）
                # 如果 JSONL 中有明確的 index 欄位，請改用: index = data.get('index', line_num)
                index = line_num
                question = data.get('question', '')
                index_to_question[index] = question
            except json.JSONDecodeError:
                print(f"⚠️ 無法解析第 {line_num + 1} 行")
                continue
    
    print(f"✅ 成功讀取 {len(index_to_question)} 筆 question 資料")
    
except FileNotFoundError:
    print("❌ 找不到 note_icd_data.jsonl 檔案")
    index_to_question = {}
except Exception as e:
    print(f"❌ 讀取 JSONL 時發生錯誤: {e}")
    index_to_question = {}

print("="*80)

# 處理 CSV 檔案
for csv_file in csv_files:
    try:
        # 讀取 CSV
        df = pd.read_csv(csv_file, encoding='utf-8')
        
        # 篩選符合條件的資料
        filtered_df = df[df['accuracy_reward'] >= THRESHOLD].copy()
        
        # 如果有資料，加入列表中
        if not filtered_df.empty:
            filtered_dfs.append(filtered_df)
            
    except Exception as e:
        # 這裡不需重複報錯，因為上方統計時已經檢查過
        pass

# 合併所有篩選後的資料
if filtered_dfs:
    final_result_df = pd.concat(filtered_dfs, ignore_index=True)
    
    # 根據 index 欄位添加 question
    if 'index' in final_result_df.columns:
        final_result_df['question'] = final_result_df['index'].map(index_to_question)
        
        # 檢查是否有找不到對應 question 的 index
        missing_mask = final_result_df['question'].isna()
        missing_questions = missing_mask.sum()
        
        if missing_questions > 0:
            # 找出所有缺少 question 的 index
            missing_indices = final_result_df.loc[missing_mask, 'index'].tolist()
            
            print(f"⚠️ 有 {missing_questions} 筆資料找不到對應的 question")
            print(f"缺少 question 的 index: {missing_indices}")
            
            # 如果數量太多，只顯示前 20 個
            if len(missing_indices) > 20:
                print(f"（僅顯示前 20 個）: {missing_indices[:20]}")
            
            # 將找不到的設為空字串
            final_result_df['question'] = final_result_df['question'].fillna('')
    else:
        print("⚠️ CSV 中沒有 'index' 欄位，無法對應 question")
        final_result_df['question'] = ''

    
    # 設定輸出檔名 (例如: 0.9_acc_result.csv)
    output_filename = f"gt_{THRESHOLD}_result.csv"
    
    # 儲存為 CSV (使用 utf-8-sig 以便 Excel 正確開啟中文)
    final_result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"✅ 成功儲存檔案: {output_filename}")
    print(f"📊 最終檔案包含筆數: {len(final_result_df)}")
    print(f"📝 成功添加 question 欄位")
else:
    print("⚠️ 沒有符合條件的資料，未產生新檔案。")

print("="*80)

正在讀取 note_icd_data.jsonl...
❌ 找不到 note_icd_data.jsonl 檔案
⚠️ 有 1044 筆資料找不到對應的 question
缺少 question 的 index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 2